In [1]:
import sys
sys.path.append('../')

from activation import *
from loss import *
from linear import * 
from encapsulage import *
from utils import *

import matplotlib.pyplot as plt
import random
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from keras.datasets import mnist
import seaborn as sns
from sklearn.metrics import confusion_matrix

import numpy as np
from keras.datasets import mnist
from sklearn.preprocessing import OneHotEncoder

def onehot(x): 
    out = [0] * 10
    out[x] = 1
    return out

In [3]:


# Charger les données MNIST
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Vectoriser les images et les normaliser
X_train = X_train.reshape(60000, 784).astype('float32') / 255
X_test = X_test.reshape(10000, 784).astype('float32') / 255

# Sélectionner un sous-ensemble
X_train = X_train[:2000]
X_test = X_test[:1000]

# Paramétrer le modèle
d = X_train.shape[1]
d_prime = 128  # Augmenter la dimension de la couche cachée pour plus de capacité d'apprentissage
d_prime2 = 64  # Ajuster la deuxième couche cachée
out = 784
epoch = 100  # Augmenter le nombre d'époques
gradient_step = 0.001  # Augmenter le taux d'apprentissage
batchsize = 64  # Taille de lot plus petite

# Initialisation des couches
lin = Linear(d, d_prime)
lin2 = Linear(d_prime, d_prime2)
lin3 = Linear(d_prime2, d_prime)
lin4 = Linear(d_prime, out)

# Transposition des paramètres pour maintenir la symétrie
lin3._parameters = lin2._parameters.T
lin4._parameters = lin._parameters.T

# Fonctions d'activation
tan = TanH()
sig = Sigmoid()

# Fonction de perte
loss = MSELoss()

# Construction de l'auto-encodeur
Encodeur = [lin, tan, lin2, tan]
Decodeur = [lin3, tan, lin4, sig]
net = Sequentiel(Encodeur + Decodeur)

# Entraînement
optim = Optim(net, loss, gradient_step)
lloss = optim.SGD(X_train, X_train, batchsize, epoch)



100%|██████████| 100/100 [00:22<00:00,  4.37it/s]


In [16]:

# Extraire les représentations latentes
class Encoder(Sequentiel):
    def forward(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x

encoder = Encoder(Encodeur)
latent_representations = encoder.forward(X_train)

# Utiliser t-SNE pour projeter les représentations latentes en 2D
tsne = TSNE(n_components=2, random_state=42)
latent_2d = tsne.fit_transform(latent_representations)

# Visualiser les représentations latentes en 2D
plt.figure(figsize=(10, 6))
sns.scatterplot(x=latent_2d[:, 0], y=latent_2d[:, 1], hue=y_train, palette='tab10', legend='full', s=60)
plt.title('Représentations latentes en 2D avec t-SNE')
plt.show()

AttributeError: 'Encoder' object has no attribute 'layers'